In [5]:
import pandas as pd
import numpy as np
import wbdata

In [6]:
def wb_clear(df, x):
    df.index = df.index.set_levels(df.index.levels[1].astype(int), level=1)
    df.index = df.index.set_levels(df.index.levels[0].str.lower(), level=0)
    df = df[df.index.get_level_values(1) >= 2006]
    df = df[df.loc[:, x].notna()]
    df.index = df.index.set_names('year', level=1)
    return df

In [32]:
def add_wb_var(code, var_name, join_path='raw_data/master.csv'):
    '''
    Get one additional variable from WB api and adds to data in master.csv

    Parameters
    ------------------
    code: str
        Identifier for the data in WB api formatte as XX.XX.XX
    var_name: str
        Variable name that will be added to return data. Can be anything.
    join_path: str, optional
        Change base file for joining.
    
    Returns
    ------------------
    pandas.Dataframe
        Dataframe containing data from join_path and new variable from WB
    '''
    base = pd.read_csv(join_path)
    x = wb_clear(wbdata.get_dataframe({code:var_name}), var_name)
    all_df = base.set_index(['country', 'year'])
    all_df = all_df.join(x).reset_index()
    
    all_df = all_df[(all_df.year == 2008) | (all_df.year == 2018)]

    m_row = [all_df.loc[i].isna().mean() > .15 for i in all_df.index]
    all_df = all_df.drop(np.array(all_df.index)[m_row], axis=0)

    m_col = [all_df.loc[:,i].isna().mean() > .15 for i in all_df.columns]
    all_df = all_df.drop(np.array(all_df.columns)[m_col], axis=1)

    no_data_country = np.array(list(all_df.groupby('country') \
        .groups.keys()))[list(all_df.groupby('country').count().min(axis=1) != 2)]
    all_df = all_df[~all_df.country.isin(no_data_country)]

    var_n = 'Jippikayjei you just successfully added one new variable!' if all_df.shape[1] > base.shape[1] else 'Damn it again too many empty values, just keep going!'
    country_n = 'You were able to keep all of the countries from the base data!' if all_df.shape[0] == base.shape[0] else 'Unfortunately you however lost some sample countries with:('
    
    print(f"{list(zip(all_df.columns, [all_df.loc[:,i].isna().sum() for i in all_df.columns]))}\n\n{all_df.shape}\n\n{var_n}\n\n{country_n}")
    
    return all_df

In [34]:
df = add_wb_var('TX.MNF.TECH.ZS.UN', 'tech_exp', join_path='raw_data/small_master_base.csv')

[('country', 0), ('year', 0), ('Unnamed: 0', 0), ('Unnamed: 0.1', 0), ('PR', 0), ('CL', 0), ('Status', 0), ('DD regime', 0), ('DD category', 0), ('Monarchy', 0), ('Commonwealth', 0), ('Democracy', 0), ('Presidential', 0), ('President name', 0), ('President accesion', 0), ('President birthyear', 0), ('Interim phase (more than 2 Presidents/year=1)', 0), ('Female president (0: No; 1: Yes)', 0), ('Colony', 0), ('Communist', 0), ('Regime change lag', 0), ('spatial democracy', 0), ('countrycode', 0), ('No. of chambers in parliament', 0), ('proportional voting', 0), ('Election system', 0), ('No. of members in lower house', 0), ('New constitution', 0), ('fullsuffrage', 0), ('electoral', 0), ('spatial electoral', 0), ('alternation', 0), ('multiparty', 0), ('free and fair election', 0), ('parliamentary election year', 0), ('postponed election', 0), ('Anticorruption policy', 0), ('Civil rights', 0), ('Civil society participation', 0), ('Civil society traditions', 0), ('Cleavage conflict managemen

In [35]:
#df.to_csv('raw_data/small_master.csv')
#df.to_csv('raw_data/small_master_base.csv')
df.to_csv('raw_data/small_master_c.csv')